# 1. Using Estimators for MNIST hand-written digit classification
## Basic imports

In [7]:
import tensorflow as tf
import tensorflow_datasets as tfds
import numpy as np

## 1. Specify the hyperparameters
- Reminder: steps_per_epoch determines the number of iterations in each epoch, which is needed for infinitely repeated datasets

In [8]:
BUFFER_SIZE = 10000
BATCH_SIZE = 64
NUM_EPOCHS = 20
steps_per_epoch = np.ceil(60000 / BATCH_SIZE)

## 2. Define a helper function that will preprocess the input image and its label

In [9]:
def preprocess(item):
    image = item['image']
    label = item['label']
    image = tf.image.convert_image_dtype(
        image, tf.float32)
    image = tf.reshape(image, (-1,))

    return {'image-pixels':image}, label[..., tf.newaxis]

## 3. Step 1: Defining the input functions (one for training and one for evaluation)

In [10]:
## define the input function for training
def train_input_fn():
    datasets = tfds.load(name='mnist')
    mnist_train = datasets['train']

    dataset = mnist_train.map(preprocess)
    dataset = dataset.shuffle(BUFFER_SIZE)
    dataset = dataset.batch(BATCH_SIZE)
    return dataset.repeat()

## define input-function for evaluation:
def eval_input_fn():
    datasets = tfds.load(name='mnist')
    mnist_test = datasets['test']
    dataset = mnist_test.map(preprocess).batch(BATCH_SIZE)
    return dataset

## 4. Step 2: Define the feature columns
- The dictionary of features has only one key, 'image-pixels'

In [11]:
image_feature_column = tf.feature_column.numeric_column(
    key='image-pixels', shape=(28*28))

## 5. Step 3: Instantiate the estimator
- We specify the number of classes (remember that MNIST consists of 10 different digits, 0-9) using the argument n_classes

In [12]:
dnn_classifier = tf.estimator.DNNClassifier(
    feature_columns=[image_feature_column],
    hidden_units=[32, 16],
    n_classes=10,
    model_dir='models/mnist-dnn/')

INFO:tensorflow:Using default config.
INFO:tensorflow:Using config: {'_model_dir': 'models/mnist-dnn/', '_tf_random_seed': None, '_save_summary_steps': 100, '_save_checkpoints_steps': None, '_save_checkpoints_secs': 600, '_session_config': allow_soft_placement: true
graph_options {
  rewrite_options {
    meta_optimizer_iterations: ONE
  }
}
, '_keep_checkpoint_max': 5, '_keep_checkpoint_every_n_hours': 10000, '_log_step_count_steps': 100, '_train_distribute': None, '_device_fn': None, '_protocol': None, '_eval_distribute': None, '_experimental_distribute': None, '_experimental_max_worker_delay_secs': None, '_session_creation_timeout_secs': 7200, '_service': None, '_cluster_spec': <tensorflow.python.training.server_lib.ClusterSpec object at 0x000001B6DDEB1608>, '_task_type': 'worker', '_task_id': 0, '_global_id_in_cluster': 0, '_master': '', '_evaluation_master': '', '_is_chief': True, '_num_ps_replicas': 0, '_num_worker_replicas': 1}


## 6. Step 4.1: Train the estimator

In [13]:
dnn_classifier.train(
    input_fn=train_input_fn,
    steps=NUM_EPOCHS * steps_per_epoch)

693, step = 27660 (0.866 sec)
INFO:tensorflow:global_step/sec: 109.065
INFO:tensorflow:global_step/sec: 109.065
INFO:tensorflow:loss = 0.21514598, step = 27760 (0.919 sec)
INFO:tensorflow:loss = 0.21514598, step = 27760 (0.919 sec)
INFO:tensorflow:global_step/sec: 111.532
INFO:tensorflow:global_step/sec: 111.532
INFO:tensorflow:loss = 0.49585629, step = 27860 (0.892 sec)
INFO:tensorflow:loss = 0.49585629, step = 27860 (0.892 sec)
INFO:tensorflow:global_step/sec: 113.802
INFO:tensorflow:global_step/sec: 113.802
INFO:tensorflow:loss = 0.25044176, step = 27960 (0.880 sec)
INFO:tensorflow:loss = 0.25044176, step = 27960 (0.880 sec)
INFO:tensorflow:global_step/sec: 221.169
INFO:tensorflow:global_step/sec: 221.169
INFO:tensorflow:loss = 0.29258835, step = 28060 (0.452 sec)
INFO:tensorflow:loss = 0.29258835, step = 28060 (0.452 sec)
INFO:tensorflow:global_step/sec: 79.6753
INFO:tensorflow:global_step/sec: 79.6753
INFO:tensorflow:loss = 0.3199266, step = 28160 (1.255 sec)
INFO:tensorflow:loss 

## 7. Step 4.2: Evaluate the estimator

In [14]:
eval_result = dnn_classifier.evaluate(
    input_fn=eval_input_fn)

print(eval_result)

INFO:tensorflow:Calling model_fn.
INFO:tensorflow:Calling model_fn.
INFO:tensorflow:Done calling model_fn.
INFO:tensorflow:Done calling model_fn.
INFO:tensorflow:Starting evaluation at 2020-07-27T02:25:12Z
INFO:tensorflow:Starting evaluation at 2020-07-27T02:25:12Z
INFO:tensorflow:Graph was finalized.
INFO:tensorflow:Graph was finalized.
INFO:tensorflow:Restoring parameters from models/mnist-dnn/model.ckpt-37520
INFO:tensorflow:Restoring parameters from models/mnist-dnn/model.ckpt-37520
INFO:tensorflow:Running local_init_op.
INFO:tensorflow:Running local_init_op.
INFO:tensorflow:Done running local_init_op.
INFO:tensorflow:Done running local_init_op.
INFO:tensorflow:Finished evaluation at 2020-07-27-02:25:14
INFO:tensorflow:Finished evaluation at 2020-07-27-02:25:14
INFO:tensorflow:Saving dict for global step 37520: accuracy = 0.9111, average_loss = 0.32053024, global_step = 37520, loss = 0.3200508
INFO:tensorflow:Saving dict for global step 37520: accuracy = 0.9111, average_loss = 0.32

# 2. Creating a custom Estimator from an existing Keras model
- Such a conversion allows us to access the strengths of Estimators, such as distributed training and automatic checkpointing
- In addition, it will make it easy for others to use this model, and particularly to avoid confusions in interpreting the input features by specifying the feature columns and the input function

## 1. Generate the XOR data and split into training and validation datasets

In [15]:
tf.random.set_seed(1)
np.random.seed(1)

## Create the data
x = np.random.uniform(low=-1, high=1, size=(200, 2))
y = np.ones(len(x))
y[x[:, 0] * x[:, 1]<0] = 0

x_train = x[:100, :]
y_train = y[:100]
x_valid = x[100:, :]
y_valid = y[100:]

## 2. Step 1: Define the input functions
- The key name for the input features must match with we define in the input layer of our model in Step 3

In [16]:
def train_input_fn(x_train, y_train, batch_size=8):
    dataset = tf.data.Dataset.from_tensor_slices(
        ({'input-features':x_train}, y_train.reshape(-1, 1)))

    # Shuffle, repeat, and batch the examples.
    return dataset.shuffle(100).repeat().batch(batch_size)

def eval_input_fn(x_test, y_test=None, batch_size=8):
    if y_test is None:
        dataset = tf.data.Dataset.from_tensor_slices(
            {'input-features':x_test})
    else:
        dataset = tf.data.Dataset.from_tensor_slices(
            ({'input-features':x_test}, y_test.reshape(-1, 1)))

    # Shuffle, repeat, and batch the examples.
    return dataset.batch(batch_size)

## 3. Define the feature columns

In [17]:
features = [
    tf.feature_column.numeric_column(
        key='input-features:', shape=(2,))
]
    
features

[NumericColumn(key='input-features:', shape=(2,), default_value=None, dtype=tf.float32, normalizer_fn=None)]

## 4. Step 3: Create the estimator - convert from a Keras model
- Before converting the model, we need to compile it

In [18]:
model = tf.keras.Sequential([
    tf.keras.layers.Input(shape=(2,), name='input-features'),
    tf.keras.layers.Dense(units=4, activation='relu'),
    tf.keras.layers.Dense(units=4, activation='relu'),
    tf.keras.layers.Dense(units=4, activation='relu'),
    tf.keras.layers.Dense(1, activation='sigmoid')
])

model.summary()

model.compile(optimizer=tf.keras.optimizers.SGD(),
              loss=tf.keras.losses.BinaryCrossentropy(),
              metrics=[tf.keras.metrics.BinaryAccuracy()])

my_estimator = tf.keras.estimator.model_to_estimator(
    keras_model=model,
    model_dir='models/estimator-for-XOR/')

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense (Dense)                (None, 4)                 12        
_________________________________________________________________
dense_1 (Dense)              (None, 4)                 20        
_________________________________________________________________
dense_2 (Dense)              (None, 4)                 20        
_________________________________________________________________
dense_3 (Dense)              (None, 1)                 5         
Total params: 57
Trainable params: 57
Non-trainable params: 0
_________________________________________________________________
INFO:tensorflow:Using default config.
INFO:tensorflow:Using default config.
INFO:tensorflow:Using the Keras model provided.
INFO:tensorflow:Using the Keras model provided.
INFO:tensorflow:Using config: {'_model_dir': 'models/estimator-for-XOR/', '_tf_random_

## 5. Step 4.1: Train the estimator

In [19]:
num_epochs = 200
batch_size = 2
steps_per_epoch = np.ceil(len(x_train) / batch_size)

my_estimator.train(
    input_fn=lambda: train_input_fn(x_train, y_train, batch_size),
    steps=num_epochs * steps_per_epoch)

40 sec)
INFO:tensorflow:global_step/sec: 733.361
INFO:tensorflow:global_step/sec: 733.361
INFO:tensorflow:loss = 0.005618953, step = 10400 (0.139 sec)
INFO:tensorflow:loss = 0.005618953, step = 10400 (0.139 sec)
INFO:tensorflow:global_step/sec: 720.946
INFO:tensorflow:global_step/sec: 720.946
INFO:tensorflow:loss = 0.0034036785, step = 10500 (0.138 sec)
INFO:tensorflow:loss = 0.0034036785, step = 10500 (0.138 sec)
INFO:tensorflow:global_step/sec: 738.548
INFO:tensorflow:global_step/sec: 738.548
INFO:tensorflow:loss = 0.005767776, step = 10600 (0.134 sec)
INFO:tensorflow:loss = 0.005767776, step = 10600 (0.134 sec)
INFO:tensorflow:global_step/sec: 732.938
INFO:tensorflow:global_step/sec: 732.938
INFO:tensorflow:loss = 0.09222086, step = 10700 (0.135 sec)
INFO:tensorflow:loss = 0.09222086, step = 10700 (0.135 sec)
INFO:tensorflow:global_step/sec: 874.714
INFO:tensorflow:global_step/sec: 874.714
INFO:tensorflow:loss = 0.19148211, step = 10800 (0.114 sec)
INFO:tensorflow:loss = 0.19148211,

## 6. Step 4.2: Evaluate the estimator

In [20]:
my_estimator.evaluate(
    input_fn=lambda: eval_input_fn(x_valid, y_valid, batch_size))

INFO:tensorflow:Calling model_fn.
INFO:tensorflow:Calling model_fn.
INFO:tensorflow:Done calling model_fn.
INFO:tensorflow:Done calling model_fn.
INFO:tensorflow:Starting evaluation at 2020-07-27T02:25:32Z
INFO:tensorflow:Starting evaluation at 2020-07-27T02:25:32Z
INFO:tensorflow:Graph was finalized.
INFO:tensorflow:Graph was finalized.
INFO:tensorflow:Restoring parameters from models/estimator-for-XOR/model.ckpt-20000
INFO:tensorflow:Restoring parameters from models/estimator-for-XOR/model.ckpt-20000
INFO:tensorflow:Running local_init_op.
INFO:tensorflow:Running local_init_op.
INFO:tensorflow:Done running local_init_op.
INFO:tensorflow:Done running local_init_op.
INFO:tensorflow:Finished evaluation at 2020-07-27-02:25:32
INFO:tensorflow:Finished evaluation at 2020-07-27-02:25:32
INFO:tensorflow:Saving dict for global step 20000: binary_accuracy = 0.96, global_step = 20000, loss = 0.0754489
INFO:tensorflow:Saving dict for global step 20000: binary_accuracy = 0.96, global_step = 20000,

{'binary_accuracy': 0.96, 'loss': 0.0754489, 'global_step': 20000}